In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from fake_useragent import UserAgent
from time import sleepπ
from random import randint
import warnings

import re

### IMDB 250 
https://www.imdb.com/search/title/?groups=top_250&sort=user_rating

In [2]:
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36"}
rank = []
title = []
year = []
runtime = []
genre = []
directors =[]
rating = []

url = 'https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&start='
pages = [1, 51, 101, 151, 201]




In [3]:
for page in pages:
    page = requests.get(url + str(page) + '&ref_=adv_nxt', headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    a = pd.DataFrame(soup.find_all('p', class_=''))
    a.columns=['director']
    for i in range(1,51):
        b= a['director'][2*i].get_text().split('\n')[2:5]
        directors.append(b) 
   
    for movie in soup.find_all('div', class_='lister-item-content'):
        rank.append(movie.find('span', class_='lister-item-index').text)
        title.append(movie.find('h3', class_='lister-item-header').a.text)
        year.append(movie.find('span', class_='lister-item-year text-muted unbold').text)
        runtime.append(movie.find('span', class_='runtime').text)
        genre.append(movie.find('span', class_='genre').text)
        rating.append(movie.find('div', class_='inline-block ratings-imdb-rating').text)
    
        

/opt/homebrew/Caskroom/miniforge/base/envs/ps/lib/python3.9/site-packages/pandas/core/internals/construction.py:540: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


In [4]:
result= {"rank" : rank, "title" : title, "year" : year, "runtime" : runtime, "genre" : genre, "director":directors, "rating" : rating}
df=pd.DataFrame(result)

df['rank']=df['rank'].str.replace('.', '')
df['year']=df['year'].str.extract('(\d+)', expand=False)
df['runtime']=df['runtime'].str.replace(' min','')
df['genre']=df['genre'].str.replace('\n','')
df['rating']=df['rating'].str.replace('\n','').astype(float)

director = []

for i in range(0,250):
    director_list = [abc for abc in df['director'][i] if abc != '    Stars:' and abc != '| ']
    
    director.append(director_list)
director
df['director'] = director
for i in range(0,250):
   df['director'][i][0] = df['director'][i][0].replace(', ', '')
   for j in range(0,250):
        try:
           df['director'][j][1] = df['director'][j][1].replace(', ', '')
        except:
           pass
df['director'] = df['director'].apply(lambda x: ', '.join(x))
df

/var/folders/c_/xfclsffs46x_pr2y70z2qfsm0000gn/T/ipykernel_59958/885374802.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['rank']=df['rank'].str.replace('.', '')


,rank,title,year,runtime,genre,director,rating
0,1,Jai Bhim,2021,164,"Crime, Drama",T.J. Gnanavel,9.3
1,2,The Shawshank Redemption,1994,142,Drama,Frank Darabont,9.3
2,3,Daeboo,1972,175,"Crime, Drama",Francis Ford Coppola,9.2
3,4,The Dark Knight,2008,152,"Action, Crime, Drama",Christopher Nolan,9.0
4,5,The Godfather: Part II,1974,202,"Crime, Drama",Francis Ford Coppola,9.0
...,...,...,...,...,...,...,...
245,246,Mr. Smith Goes to Washington,1939,129,"Comedy, Drama",Frank Capra,8.1
246,247,Gone with the Wind,1939,238,"Drama, History, Romance","Victor Fleming, George Cukor, Sam Wood",8.1
247,248,It Happened One Night,1934,105,"Comedy, Romance",Frank Capra,8.1
248,249,La passion de Jeanne d'Arc,1928,110,"Biography, Drama, History",Carl Theodor Dreyer,8.1


In [5]:
#1.1) 가장 빈도가 높은 상위 3개 연도
df['year'].value_counts().head(3)


1995    8
1957    7
2019    7
Name: year, dtype: int64

In [6]:
# 1.2) 연도별 rating의 평균을 이용해서 평균치가 가장높은 연도와 평균치를 구하시오
df.groupby('year').mean()['rating'].sort_values(ascending=False).head(1)


year
1972    9.2
Name: rating, dtype: float64

In [7]:
# 1.3) 가장빈도가 높은 장르는 무엇이며 몇편인가?(단.여러쟝르조합은 하나의 별도의 쟝르로 구분한다)
df['genre'].value_counts().head(1)

Drama                21
Name: genre, dtype: int64

In [8]:
# 1.4) 6편이상을 제작한 감독은 누구인가?
directors_all = list()
for i in range(0,250):
    directors_all += ''.join(df['director'][i]).split(', ')
    
directors_all = pd.DataFrame(directors_all)
directors_all.columns = ['director']  
directors_all['director'].value_counts().head(6)



Stanley Kubrick      7
Christopher Nolan    7
Martin Scorsese      7
Akira Kurosawa       7
Alfred Hitchcock     6
Steven Spielberg     6
Name: director, dtype: int64

### 네이버 영화랭킹 
https://movie.naver.com/movie/sdb/rank/rmovie.naver?sel=pnt&date=20201108

In [9]:
url = 'https://movie.naver.com/movie/sdb/rank/rmovie.naver?sel=pnt&date=20201108'
req = requests.get(url, headers=headers)
soup = BeautifulSoup(req.text, 'html.parser')



In [10]:
# 2.1) 영화명, 평점, 상세url 데이터프레임
titles = []
ratings = []
urls = []

links = soup.select('.title') 
for link in links: 
    add = 'https://movie.naver.com'+ link.find('a')['href']
    urls.append(add)

for title in links: 
    title = title.get_text() 
    title = re.sub('\n','',title)
    titles.append(title)

ratingfind = soup.select('.point')
for rating in ratingfind:
    rating = rating.get_text()
    rating = re.sub('\n','',rating)
    ratings.append(rating)

# result = {"title" : titles, "rating" : ratings, "url" : urls}
# df2 = pd.DataFrame(result)
# df2['rating'] = df2['rating'].str.replace('\n','')
# df2['rating'] = df2['rating'].str.replace('.','')
# df2['rating'] = df2['rating'].astype(float)/100
# df2

movie_list = zip(titles, ratings, urls)
movie_info_list =[]
for movie in movie_list:
    movie_dict ={
    "title":movie[0],
    "rating":movie[1],
    "url":movie[2]
    }
    movie_info_list.append(movie_dict)
movie_info_list = pd.DataFrame(movie_info_list)
movie_info_list


,title,rating,url
0,그린 북,9.59,https://movie.naver.com/movie/bi/mi/basic.nave...
1,가버나움,9.59,https://movie.naver.com/movie/bi/mi/basic.nave...
2,디지몬 어드벤처 라스트 에볼루션 : 인연,9.57,https://movie.naver.com/movie/bi/mi/basic.nave...
3,먼 훗날 우리,9.54,https://movie.naver.com/movie/bi/mi/basic.nave...
4,베일리 어게인,9.53,https://movie.naver.com/movie/bi/mi/basic.nave...
5,부활: 그 증거,9.53,https://movie.naver.com/movie/bi/mi/basic.nave...
6,아일라,9.50,https://movie.naver.com/movie/bi/mi/basic.nave...
7,원더,9.50,https://movie.naver.com/movie/bi/mi/basic.nave...
8,포드 V 페라리,9.49,https://movie.naver.com/movie/bi/mi/basic.nave...
9,당갈,9.48,https://movie.naver.com/movie/bi/mi/basic.nave...


In [11]:
# 개별작품사이트에서 아래의 변수를 추출 data frame으로 만들기
# 영화명, 관람객평점, 쟝르, 영화국적, 상영시간, 개봉연도
score_list = []
type_list = []
country_list = []
run_time_list = []
open_time_list = []
for url_i in urls:
    html_new = requests.get(url_i, headers=headers).text
    soup_new = BeautifulSoup(html_new, 'html.parser')
    score = soup_new.select(".star_score > span > span")[0].text
    type = soup_new.select(".info_spec dd > p > span")[0].text
    type = re.sub('\t|\r|\n','',type)
    country = soup_new.select(".info_spec dd > p > span")[1].text
    country = re.sub('\t|\r|\n','',country)
    try:
        run_time = soup_new.select(".info_spec dd > p > span")[2].text
        run_time = re.sub('\t|\r|\n','',run_time)
    except:
        run_time = None
    
    try:
        open_time = soup_new.select(".info_spec dd > p > span")[3].text
        open_time = re.sub('\t|\r|\n','',open_time)
    
    except:
        open_time = None

    score_list.append(score)
    type_list.append(type)
    country_list.append(country)
    run_time_list.append(run_time)
    open_time_list.append(open_time)
    

result = {"score" : score_list, "type" : type_list, "country" : country_list, "run_time" : run_time_list, "open_time" : open_time_list}
df3 = pd.DataFrame(result)
movie_info_list = pd.concat([movie_info_list, df3], axis=1)
movie_info_list

,title,rating,url,score,type,country,run_time,open_time
0,그린 북,9.59,https://movie.naver.com/movie/bi/mi/basic.nave...,관람객 평점 9.55점,드라마,미국,130분,2019.01.09 개봉
1,가버나움,9.59,https://movie.naver.com/movie/bi/mi/basic.nave...,관람객 평점 9.54점,드라마,"레바논, 프랑스",126분,2019.01.24 개봉
2,디지몬 어드벤처 라스트 에볼루션 : 인연,9.57,https://movie.naver.com/movie/bi/mi/basic.nave...,관람객 평점 없음,"애니메이션, 모험",일본,114분,2021.02.17 개봉
3,먼 훗날 우리,9.54,https://movie.naver.com/movie/bi/mi/basic.nave...,관람객 평점 없음,"드라마, 멜로/로맨스",중국,120분,None
4,베일리 어게인,9.53,https://movie.naver.com/movie/bi/mi/basic.nave...,관람객 평점 9.42점,"모험, 코미디, 드라마",미국,100분,2018.11.22 개봉
5,부활: 그 증거,9.53,https://movie.naver.com/movie/bi/mi/basic.nave...,관람객 평점 9.82점,다큐멘터리,한국,99분,2020.10.08 개봉
6,아일라,9.50,https://movie.naver.com/movie/bi/mi/basic.nave...,관람객 평점 9.13점,"드라마, 전쟁","한국, 터키",123분,2018.06.21 개봉
7,원더,9.50,https://movie.naver.com/movie/bi/mi/basic.nave...,관람객 평점 9.43점,드라마,미국,113분,"2021.02.11 재개봉, 2017.12.27 개봉"
8,포드 V 페라리,9.49,https://movie.naver.com/movie/bi/mi/basic.nave...,관람객 평점 9.31점,"액션, 드라마",미국,152분,2019.12.04 개봉
9,당갈,9.48,https://movie.naver.com/movie/bi/mi/basic.nave...,관람객 평점 9.62점,"드라마, 액션",인도,161분,2018.04.25 개봉


In [12]:
# 2.2)장르가 [다큐멘터리]인 영화의 제목은 무엇인가?
df3[df3['type'] == '다큐멘터리']

,score,type,country,run_time,open_time
5,관람객 평점 9.82점,다큐멘터리,한국,99분,2020.10.08 개봉
10,관람객 평점 9.67점,다큐멘터리,미국,121분,2019.07.25 개봉
44,관람객 평점 없음,다큐멘터리,한국,89분,2020.09.24 개봉


In [13]:
# 2.3) 영화국적의 빈도가 가장 높은 나라는?
df3['country'].value_counts().head(1)



미국    25
Name: country, dtype: int64

In [14]:
# 2.4) 개봉연도기준으로 관람객평점의 평균이 가장높은 [1]개봉연도와 [2]평균평점을 구하시오 여기서 관람객평점 결측치가 있는 연도는 데이터에서 제외시킨뒤 계산하시오
find = '관람객 평점 없음'
new_movie_info_list=movie_info_list.query('score not in @find')
new_movie_info_list['score'] = new_movie_info_list['score'].str.replace("[ㄱ-ㅎ ㅏ-ㅣ 가-힣]","",regex=True)

#개봉연도기준으로 관람객평점의 평균이 가장높은 [1]개봉연도와 [2]평균평점을 구하시오
new_movie_info_list.groupby('open_time')['score'].mean().sort_values(ascending=False).head(1)
#2019.09.05 재개봉,  2002.04.05 개봉    10.0

/var/folders/c_/xfclsffs46x_pr2y70z2qfsm0000gn/T/ipykernel_59958/2376592996.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movie_info_list['score'] = new_movie_info_list['score'].str.replace("[ㄱ-ㅎ ㅏ-ㅣ 가-힣]","",regex=True)


open_time
2019.09.05 재개봉,  2002.04.05 개봉    10.0
Name: score, dtype: float64

In [15]:
# 2.5) 쟝르 기준으로 상영시간의 평균이 가장 긴 쟝르를 구하시오
movie_info_list['run_time'] = movie_info_list['run_time'].str.replace("[ㄱ-ㅎ ㅏ-ㅣ 가-힣]","",regex=True)
movie_info_list['run_time'] = movie_info_list['run_time'].astype(int)

movie_info_list.groupby('type')['run_time'].mean().sort_values(ascending=False).head(1)
# 여러쟝르 또는 영화국적의 조합은 하나의 별도의 쟝르 또는 국적으로 보기 때문에 방영시간 젤높으며 쟝르가 1개인 반지의제왕 왕의귀환이 선택됨

type
판타지, 모험, 액션, 전쟁    263.0
Name: run_time, dtype: float64